In [ ]:
import asyncio
import os
import subprocess
import sys
import pandas as pd
import re
import json

# --- 1. УСТАНОВКА БИБЛИОТЕК ---
def install_libs():
    try:
        import playwright
        import bs4
    except ImportError:
        print("⏳ Устанавливаем библиотеки...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "playwright", "beautifulsoup4", "pandas"])
        os.system("playwright install chromium")
        os.system("playwright install-deps")
        print("✅ Библиотеки готовы.")

install_libs()

from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

BASE_URL_SOBOR = "https://sobor39.ru"

# --- 2. ПАРСЕР СОБОРА ---
async def scrape_sobor_strict(page):
    target_url = "https://sobor39.ru/events/concerts/night/"
    print(f"⛪ Заходим на {target_url}...")
    
    try:
        await page.goto(target_url, timeout=90000, wait_until='domcontentloaded')
        await page.wait_for_timeout(5000)
    except Exception as e:
        print(f"⚠️ Предупреждение загрузки: {e}. Пробуем работать с тем, что есть.")

    # --- ПРОКРУТКА ---
    print("⬇️ Начинаем прокрутку для подгрузки событий...")
    
    for i in range(10):  
        await page.mouse.wheel(0, 5000)
        await page.wait_for_timeout(3000)
        
        try:
            more_btns = page.locator("text=/Показать ещ[ёе]|Загрузить/i")
            if await more_btns.count() > 0:
                if await more_btns.first.is_visible():
                    await more_btns.first.click()
                    await page.wait_for_timeout(2000)
        except:
            pass
            
        if i % 2 == 0:
            print(f"   ...прокрутка {i+1}/10")

    content = await page.content()
    soup = BeautifulSoup(content, 'html.parser')
    
    events_list = []
    unique_keys = set()
    
    items = soup.find_all('div', class_='list-item')
    print(f"🔍 Найдено блоков .list-item: {len(items)}")

    for item in items:
        try:
            # 1. ЗАГОЛОВОК
            title_tag = item.find('div', class_='list-t')
            if not title_tag:
                continue
            title = title_tag.get_text(strip=True)
            
            # 2. ДАТА (Раздельные классы)
            date_raw = "Unknown"
            day_node = item.find('div', class_='list-day')
            month_node = item.find('div', class_='list-month')
            time_node = item.find('div', class_='list-time')
            
            if day_node and month_node:
                d_txt = day_node.get_text(strip=True)
                m_txt = month_node.get_text(strip=True)
                t_txt = time_node.get_text(strip=True) if time_node else ""
                date_raw = f"{d_txt} {m_txt} {t_txt}".strip()
            
            # 3. ССЫЛКА И СТАТУС
            url = ""
            status = "available"
            
            other_block = item.find('div', class_='list-other')
            if other_block:
                btn = other_block.find('a', href=True)
                if btn:
                    link = btn['href']
                    url = link if link.startswith('http') else f"{BASE_URL_SOBOR}{link}"
                    btn_text = btn.get_text(strip=True).upper()
                    if "SOLD" in btn_text or "ПРОДАНО" in btn_text:
                        status = "sold_out"
            
            if not url:
                status = "unknown"

            # 4. КАРТИНКА
            photo_url = ""
            img_block = item.find('div', class_='list-img')
            if img_block:
                img = img_block.find('img')
                if img and img.get('src'):
                    src = img['src']
                    if 'svg' not in src:
                        photo_url = src if src.startswith('http') else f"{BASE_URL_SOBOR}{src}"

            # 5. ОПИСАНИЕ (Markdown)
            description = ""
            descr_node = item.find('div', class_='list-descr')
            if descr_node:
                # get_text(separator="\n") заменяет <br> и закрывающие теги на перенос строки
                description = descr_node.get_text(separator="\n", strip=True)

            # Дедубликация
            key = f"{date_raw}_{title}"
            if key in unique_keys:
                continue
            unique_keys.add(key)

            events_list.append({
                "title": title,
                "date_raw": date_raw,
                "ticket_status": status,
                "url": url,
                "photos": [photo_url] if photo_url else [],
                "description": description,
                "pushkin_card": False, 
                "location": "Кафедральный собор"
            })
        except Exception as e:
            print(f"⚠️ Ошибка при разборе элемента: {e}")
            continue

    print(f"✅ Успешно обработано событий: {len(events_list)}")
    return events_list

# --- ЗАПУСК ---
async def main_sobor():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(viewport={'width': 1920, 'height': 1080})
        page = await context.new_page()
        
        data = await scrape_sobor_strict(page)
        await browser.close()
        return data

result = await main_sobor()

if result:
    with open('sobor.json', 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=4)
        
    df = pd.DataFrame(result)
    if 'photos' in df.columns:
        df['photo_preview'] = df['photos'].apply(lambda x: x[0] if x else "")
    
    # Обрезаем описание для превью в консоли
    if 'description' in df.columns:
        df['desc_preview'] = df['description'].apply(lambda x: (x[:50] + '...') if len(x) > 50 else x)

    print(f"\n🎉 ИТОГ: {len(result)} событий.")
    pd.set_option('display.max_colwidth', 40)
    print(df[['date_raw', 'title', 'desc_preview']].head(10))
    
    df.drop(columns=['photos', 'desc_preview'], inplace=True, errors='ignore')
    df.to_csv('sobor.csv', index=False)
else:
    print("Данные не найдены.")